# Run Blob Detection on Pixel Heatmap to Identify Candidate Sites
Note: This is only working on inputs with EPSG CRS 4326. I may need to make it more general in the future.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from keplergl import KeplerGl
from scripts import candidate_detect

In [ ]:
kepler_config = { "version": "v1", "config": { "visState": { "layers": [ { "id": "iik903a", "type": "point", "config": { "dataId": "Candidate Peaks", "label": "Point", "color": [ 218, 0, 0 ], "columns": { "lat": "lat", "lng": "lon", "altitude": None }, "isVisible": True, "visConfig": { "radius": 20, "fixedRadius": False, "opacity": 0.99, "outline": True, "thickness": 3, "strokeColor": [ 210, 0, 0 ], "filled": False }, }, }, { "id": "kyoc7uj", "type": "point", "config": { "dataId": "Candidate Blobs", "label": "Point", "color": [ 246, 218, 0 ], "columns": { "lat": "lat", "lng": "lon", "altitude": None }, "isVisible": True, "visConfig": { "radius": 20, "fixedRadius": False, "opacity": 0.99, "outline": True, "thickness": 3, "strokeColor": [ 246, 218, 0 ], "filled": False }, }, } ], }, "mapStyle": { "styleType": "satellite", } } }

In [ ]:
model_name = '0.0.11'
name = 'lagos_v0.0.11_2019-01-01_2021-06-01mosaic-median'

In [ ]:
# Detect candidates from directory of tiles. Multiprocessed
source_dir = f'../data/model_outputs/heatmaps/{model_name}/{name}'
blobs = candidate_detect.detect_blobs_tiled(source_dir,
                           name=name,
                           model_name=model_name,
                           pred_threshold=0.6, 
                           min_sigma=3.5, 
                           max_sigma=100, 
                           area_threshold=0.0025, 
                           save=True,
                           merge_radius=0.005
                          )


In [ ]:
# Display Kepler map of points
candidate_map = KeplerGl(height=800, config=kepler_config)
candidate_map.add_data(data=blobs.copy(), name='Candidate Blobs')
candidate_map

### Other forms of candidate detection. Not used

In [ ]:
import rasterio as rs
source = rs.open(f'../data/model_outputs/heatmaps/{MODEL_NAME}/{name}.tif')

In [ ]:
peaks = candidate_detect.detect_peaks(source, name=name, min_distance=100, threshold_abs=0.95, save=False)
candidate_map.add_data(data=peaks.copy(), name='Candidate Peaks')

In [ ]:
blobs = candidate_detect.detect_blobs(source, 
                     name=name, 
                     pred_threshold=0.9, 
                     min_sigma=5, 
                     area_threshold=0.0025,
                     save=True)
candidate_map.add_data(data=blobs.copy(), name='Candidate Blobs')